In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("UDF").getOrCreate()

In [ ]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [4]:
dataframe = spark.createDataFrame([
    (0, "Mary had a little lamb"),
    (1, "Its fleece was white as snow"),
    (2, "And everywhere Mary went"),
    (3, "The lamb was sure to go")
], ["id", "Nursery Rhyme"])
dataframe.show()

+---+--------------------+
| id|       Nursery Rhyme|
+---+--------------------+
|  0|Mary had a little...|
|  1|Its fleece was wh...|
|  2|And everywhere Ma...|
|  3|The lamb was sure...|
+---+--------------------+



In [5]:
# Tokenize word
tokenizer = Tokenizer(inputCol="Nursery Rhyme", outputCol="words")
tokenizer

Tokenizer_6d2129429046

In [ ]:
# Create a function to return the length of a list
def word_list_length(word_list):
    return len(word_list)

In [7]:
# Create a user defined function 
count_tokens = udf(word_list_length, IntegerType())
count_tokens

<function __main__.word_list_length>

In [8]:
# Transform DataFrame
tokenized = tokenizer.transform(dataframe)

# Select the needed columns and don't truncate results
tokenized.select("Nursery Rhyme", "words")\
    .withColumn("tokens", count_tokens(col("words"))).show(truncate=False)

+----------------------------+-----------------------------------+------+
|Nursery Rhyme               |words                              |tokens|
+----------------------------+-----------------------------------+------+
|Mary had a little lamb      |[mary, had, a, little, lamb]       |5     |
|Its fleece was white as snow|[its, fleece, was, white, as, snow]|6     |
|And everywhere Mary went    |[and, everywhere, mary, went]      |4     |
|The lamb was sure to go     |[the, lamb, was, sure, to, go]     |6     |
+----------------------------+-----------------------------------+------+

